#**Algoritmi su grafi**#

Basato sul codice reperibile online (https://ocw.mit.edu/courses/6-0002-introduction-to-computational-thinking-and-data-science-fall-2016/resources/lecturegraphs/)

##**Classe Digraph e Graph**##
Costruiamo le classe Digraph e Graph:
Per prima cosa introduciamo le due classi fondamentali: nodo e arco

In [1]:
class Node(object):
    def __init__(self, name):
        """Assumes name is a string"""
        self.name = name
    def getName(self):
        return self.name
    def __str__(self):
        return self.name

class Edge(object):
    def __init__(self, src, dest):
        """Assumes src and dest are nodes"""
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return self.src.getName() + '->' + self.dest.getName()

Una volta definiti nodi ed archi possiamo introdurre la classe **Digraph**, che rappresenta un grafo diretto.

Digraph è caratterizzata da un singolo attributo, **edges**, che consiste in un dizionario che mappa ogni nodo alla lista dei suoi vicini (orientati). Nota bene: stiamo rappresentando il grafo come una **lista d'adiacenza** (in realtà un dizionario d'adiacenza).

In [2]:
class Digraph(object):
    """edges is a dict mapping each node to a list of
    its children"""
    def __init__(self):
        self.edges = {}
    def addNode(self, node):
        if node in self.edges:
            raise ValueError('Duplicate node')
        else:
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not (src in self.edges and dest in self.edges):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.edges
    def getNode(self, name):
        for n in self.edges:
            if n.getName() == name:
                return n
        raise NameError(name)
    def getNodeList(self):
        nodeList = []
        for node in self.edges:
          nodeList.append(node)
        return nodeList
    def __str__(self):
        result = ''
        for src in self.edges:
            for dest in self.edges[src]:
                result = result + src.getName() + '->' + dest.getName() + '\n'
        return result[:-1] #omit final newline

Basandoci sulla classe Digraph possiamo definire la classe **Graph**, che rappresenta un grafo indiretto.

Nota bene, l'unica differenza tra le due classi consiste nel metodo **addEdge**: quando un arco {u,v} viene inserito, addEdge inserisce sia l'arco diretto (u,v) che (v,u).   

In [3]:
class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

Creiamo ora una funzione che istanzia un **grafo di alcune città italiane**, collegate dall'A1, l'A4, l'A14 e l'A13. Nota, è ragionevole usare un grafo non diretto.

In [4]:
def buildCityGraph(graphType):
    g = graphType()
    cities = ('Milano', 'Bologna', 'Firenze', 'Roma', 'Pescara', 'Bari', 'Taranto', 'Torino', 'Venezia', 'Trieste', 'Napoli', 'Padova', 'Salerno', 'Potenza', 'Catanzaro', 'Reggio Calabria')
    for city in cities: #Create 11 nodes
        g.addNode(Node(city))
    g.addEdge(Edge(g.getNode('Milano'), g.getNode('Bologna')))
    g.addEdge(Edge(g.getNode('Bologna'), g.getNode('Firenze')))
    g.addEdge(Edge(g.getNode('Firenze'), g.getNode('Roma')))
    g.addEdge(Edge(g.getNode('Roma'), g.getNode('Napoli')))
    g.addEdge(Edge(g.getNode('Bologna'), g.getNode('Pescara')))
    g.addEdge(Edge(g.getNode('Pescara'), g.getNode('Bari')))
    g.addEdge(Edge(g.getNode('Bari'), g.getNode('Taranto')))
    g.addEdge(Edge(g.getNode('Torino'), g.getNode('Milano')))
    g.addEdge(Edge(g.getNode('Milano'), g.getNode('Padova')))
    g.addEdge(Edge(g.getNode('Padova'), g.getNode('Venezia')))
    g.addEdge(Edge(g.getNode('Bologna'), g.getNode('Padova')))
    g.addEdge(Edge(g.getNode('Venezia'), g.getNode('Trieste')))
    g.addEdge(Edge(g.getNode('Salerno'), g.getNode('Potenza')))
    g.addEdge(Edge(g.getNode('Potenza'), g.getNode('Catanzaro')))
    g.addEdge(Edge(g.getNode('Catanzaro'), g.getNode('Reggio Calabria')))
    return g

Istanziamo il grafo e usiamo print per verificare sia tutto in ordine

In [5]:
g = buildCityGraph(Graph)
print(g)

Milano->Bologna
Milano->Torino
Milano->Padova
Bologna->Milano
Bologna->Firenze
Bologna->Pescara
Bologna->Padova
Firenze->Bologna
Firenze->Roma
Roma->Firenze
Roma->Napoli
Pescara->Bologna
Pescara->Bari
Bari->Pescara
Bari->Taranto
Taranto->Bari
Torino->Milano
Venezia->Padova
Venezia->Trieste
Trieste->Venezia
Napoli->Roma
Padova->Milano
Padova->Venezia
Padova->Bologna
Salerno->Potenza
Potenza->Salerno
Potenza->Catanzaro
Catanzaro->Potenza
Catanzaro->Reggio Calabria
Reggio Calabria->Catanzaro


##**Depth-first search**##

Descriviamo ora una funzione che ricerca il percorso minimo tra due nodi usando DFS


In [7]:
def printPath(path):
    """Assumes path is a list of nodes"""
    result = ''
    for i in range(len(path)):
        result = result + str(path[i])
        if i != len(path) - 1:
            result = result + '->'
    return result

def DFS(graph, start, end, path, shortest, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes;
          path and shortest are lists of nodes
       Returns a shortest path from start to end in graph"""
    path = path + [start]
    if toPrint:
        print('Current DFS path:', printPath(path))
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #avoid cycles
            if shortest == None or len(path) < len(shortest):
                newPath = DFS(graph, node, end, path, shortest,
                              toPrint)
                if newPath != None:
                    shortest = newPath
        elif toPrint:
            print('Already visited', node)
    return shortest

def shortestPath_DFS(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return DFS(graph, start, end, [], None, toPrint)


Testiamo ora la DFS per trovare la distanza minima tra Torino e Napoli

In [10]:
def testSP_DFS(source, destination):
    g = buildCityGraph(Graph)
    sp = shortestPath_DFS(g, g.getNode(source), g.getNode(destination),
                      toPrint = True)
    if sp != None:
        print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
    else:
        print('There is no path from', source, 'to', destination)

testSP_DFS('Napoli', 'Torino')


Current DFS path: Napoli
Current DFS path: Napoli->Roma
Current DFS path: Napoli->Roma->Firenze
Current DFS path: Napoli->Roma->Firenze->Bologna
Current DFS path: Napoli->Roma->Firenze->Bologna->Milano
Already visited Bologna
Current DFS path: Napoli->Roma->Firenze->Bologna->Milano->Torino
Current DFS path: Napoli->Roma->Firenze->Bologna->Milano->Padova
Already visited Milano
Already visited Bologna
Already visited Firenze
Current DFS path: Napoli->Roma->Firenze->Bologna->Pescara
Already visited Bologna
Current DFS path: Napoli->Roma->Firenze->Bologna->Pescara->Bari
Already visited Pescara
Current DFS path: Napoli->Roma->Firenze->Bologna->Padova
Current DFS path: Napoli->Roma->Firenze->Bologna->Padova->Milano
Already visited Bologna
Already visited Padova
Current DFS path: Napoli->Roma->Firenze->Bologna->Padova->Venezia
Already visited Padova
Already visited Bologna
Already visited Roma
Already visited Napoli
Shortest path from Napoli to Torino is Napoli->Roma->Firenze->Bologna->Milano

##**Breadth-first search**##

Descriviamo ora una funzione che ricerca il percorso minimo tra due nodi usando BFS

In [12]:
printQueue = True

def BFS(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    initPath = [start]
    pathQueue = [initPath]
    while len(pathQueue) != 0:
        #Get and remove oldest element in pathQueue
        if printQueue:
            print('Queue:', len(pathQueue))
            for p in pathQueue:
                print(printPath(p))
        tmpPath = pathQueue.pop(0)
        if toPrint:
            print('Current BFS path:', printPath(tmpPath))
            print()
        lastNode = tmpPath[-1]
        if lastNode == end:
            return tmpPath
        for nextNode in graph.childrenOf(lastNode):
            if nextNode not in tmpPath:
                newPath = tmpPath + [nextNode]
                pathQueue.append(newPath)
    return None


Cerchiamo ora il percorso minimo tra Napoli e Torino usando BFS:

In [14]:
def shortestPath_BFS(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return BFS(graph, start, end, toPrint)

def testSP_BFS(source, destination):
    g = buildCityGraph(Graph)
    sp = shortestPath_BFS(g, g.getNode(source), g.getNode(destination),
                      toPrint = True)
    if sp != None:
        print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
    else:
        print('There is no path from', source, 'to', destination)

testSP_BFS('Napoli', 'Torino')

Queue: 1
Napoli
Current BFS path: Napoli

Queue: 1
Napoli->Roma
Current BFS path: Napoli->Roma

Queue: 1
Napoli->Roma->Firenze
Current BFS path: Napoli->Roma->Firenze

Queue: 1
Napoli->Roma->Firenze->Bologna
Current BFS path: Napoli->Roma->Firenze->Bologna

Queue: 3
Napoli->Roma->Firenze->Bologna->Milano
Napoli->Roma->Firenze->Bologna->Pescara
Napoli->Roma->Firenze->Bologna->Padova
Current BFS path: Napoli->Roma->Firenze->Bologna->Milano

Queue: 4
Napoli->Roma->Firenze->Bologna->Pescara
Napoli->Roma->Firenze->Bologna->Padova
Napoli->Roma->Firenze->Bologna->Milano->Torino
Napoli->Roma->Firenze->Bologna->Milano->Padova
Current BFS path: Napoli->Roma->Firenze->Bologna->Pescara

Queue: 4
Napoli->Roma->Firenze->Bologna->Padova
Napoli->Roma->Firenze->Bologna->Milano->Torino
Napoli->Roma->Firenze->Bologna->Milano->Padova
Napoli->Roma->Firenze->Bologna->Pescara->Bari
Current BFS path: Napoli->Roma->Firenze->Bologna->Padova

Queue: 5
Napoli->Roma->Firenze->Bologna->Milano->Torino
Napoli->Roma->

##**Componenti Connesse**##

Qui studiamo come usare quanto visto finora per studiare la connessione di un grafo.

In [ ]:
def reachableFrom(graph, start):
    """Assumes graph is a Digraph; start is a node;
      returns the list of nodes reachable from start."""
    reachableNodes = {start}
    to_visit = []
    for node in graph.childrenOf(start):
      to_visit.append(node)
    while len(to_visit) > 0:
      node = to_visit.pop()
      reachableNodes.add(node)
      for child in graph.childrenOf(node):
        if child not in reachableNodes:
            to_visit.append(child)
    return reachableNodes

def printConnectedComponent(component):
    """print nodes in component"""
    component = list(component)
    result = ''
    for i in range(len(component)):
        result = result + str(component[i])
        if i != len(component) - 1:
            result = result + ', '
    return result

In [ ]:
printConnectedComponent(reachableFrom(g,g.getNode('Napoli')))

'Bologna, Torino, Trieste, Venezia, Taranto, Firenze, Roma, Padova, Napoli, Pescara, Milano, Bari'

Possiamo 'automatizzare' questo procedimento per verificare che un grafo (indiretto) sia connesso o no. NB: Quale è il tempo di esecuzione di questo algoritmo?

In [ ]:
def isConnected(graph):
  nodes = g.getNodeList()
  if nodes == []:
    return True
  start = nodes[0]
  component = reachableFrom(g, start)
  if len(component)!= len(nodes):
    return False
  return True

Effettivamente il grafo non è connesso: presenta due componenti connesse!

In [ ]:
isConnected(g)

False